# MNIST Digits Classification - Extended PyTorch🔥CNN 

Differences to the "Simple PyTorch🔥CNN" Notebook: 
- More complex network (more convoluted layers, more filters)
- As CUDA runs into memory problems on Kaggle when predicting a whole train, validation, or test dataset, we need to use batches to compute the metrics

Subject: Building a CNN Classificator with PyTorch to classify hand-written digits.

Data: MNIST (handwritten digits) via torchvision

Procedure:
- Previewing images from dataset with pyplot's imshow()
- Neural network with torch.nn.module, torch.nn.Sequential, torch.nn.Conv2d, torch.nn.ReLU, torch.nn.MaxPool2d, torch.nn.Dropout, torch.nn.Linear, and torch.nn.init.xavier_uniform_
- Visualizing model with torchviz' make_dot()
- Training with nn.CrossEntropyLoss and torch.optim.Adam optimizer
- Visualization Loss and Accuracy with pyplot
- Visualization of the CNN Layers with pyplot's imshow()
- Good results

Others:
- Compatible with Google Colab and Kaggle as runtime
- CUDA support

Sources used:
- https://machinelearningknowledge.ai/pytorch-conv2d-explained-with-examples/
- https://www.kaggle.com/code/raghaw/mnist-cnn-external-data-perfect-score
- Probably some more, but this is an old Notebook and I forgot the sources. Please let me know if I copied your code and I will mention it.

## Bootstrap and Imports

In [1]:
import os
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Running on {DEVICE}')

# running in google colab
if 'google.colab' in str(get_ipython()):
    NUM_EPOCHS = 50
    !pip install torchviz
    BASE_PATH = './drive/MyDrive/Colab/data/'
    from google.colab import drive
    drive.mount('/content/drive')

# running interactively in kaggle
elif get_ipython().config.IPKernelApp.connection_file.startswith('/root/.local/share'):
    NUM_EPOCHS = 5
    BASE_PATH = '/kaggle/input/'
    !pip install torchviz
    
# running as background job in kaggle
elif 'SHLVL' in os.environ:
    NUM_EPOCHS = 50
    BASE_PATH = '/kaggle/input/'
    !pip install torchviz

else:
    BASE_PATH = '../data/'
    NUM_EPOCHS = 2

Running on cpu


In [2]:
import random
from tqdm.auto import tqdm
import numpy as np
from collections.abc import Callable
import locale
locale.setlocale(locale.LC_ALL, locale='')  # for thousands separator via ... print(f'{value:n}')"
import math
from itertools import islice
from collections.abc import Iterable, Generator

from IPython.display import HTML, Image
import time
import matplotlib.animation as animation
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import torch
from torch import nn
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision import utils
from torch.utils.data import DataLoader
from torch.nn.modules.loss import _Loss
from torchviz import make_dot
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import sklearn.metrics

my_seed = 123
random.seed(my_seed)
torch.manual_seed(my_seed)

## Dataset

## Load Data into Pandas

In [3]:
path_train = BASE_PATH + 'digit-recognizer/train.csv'
path_test = BASE_PATH + 'digit-recognizer/test.csv'

df_train_source = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

print(df_train_source.columns)
print(df_test.columns)

print(df_train_source.shape)
print(df_test.shape)
assert 'label' in df_train_source
assert 'label' not in df_test

FileNotFoundError: [Errno 2] No such file or directory: '../data/digit-recognizer/train.csv'

In [ ]:
# split train into train and validation
df_train_randomized = df_train_source.sample(frac=1)
NUM_VAL = int(len(df_train_randomized) * 0.15)

df_train_with_label = df_train_randomized[:-NUM_VAL]  # (35700, 785)
df_val_with_label = df_train_randomized[-NUM_VAL:]  # (6300, 785)

In [ ]:
df_train = df_train_with_label.iloc[:,1:]  # (35700, 784)
df_val = df_val_with_label.iloc[:,1:]  # (6300, 784)

ser_y_train = df_train_with_label.iloc[:,0]  # (35700,)
ser_y_val = df_val_with_label.iloc[:,0]  # (6300,)

In [ ]:
assert all(df_train.columns == df_val.columns)
assert all(df_train.columns == df_test.columns)

# no nan treatment required
assert df_train.isnull().sum().sum() == 0

## Tensorize

In [ ]:
# from flat 784 (1..255) to 28*28 pixels with 1 normalized channel (0.0..1.0)
def reshape(df: pd.DataFrame) -> np.array:  # df: (n, 784), all int64
    df = df.values.reshape(-1, 28, 28)  # (n, 28, 28), int64
    df = df.astype(np.float32)  # (n, 28, 28), float32
    # the pixels have 256 values (0.0..255.0), therefore we normalize to (0.0..1.0)
    df = df / 255.0
    return df
    
    
arr_train = reshape(df_train)  # np.array (35700, 28, 28), float32
arr_val = reshape(df_val)  # np.array (6300, 28, 28)
arr_test = reshape(df_test)  # np.array (28000, 28, 28)

In [ ]:
# tensorize
train = torch.tensor(arr_train).to(DEVICE)  # torch.float32
val = torch.tensor(arr_val).to(DEVICE)
test = torch.tensor(arr_test).to(DEVICE)

y_train = torch.tensor(ser_y_train.values).to(DEVICE)  # [35700], torch.int64
y_val = torch.tensor(ser_y_val.values).to(DEVICE)  # [6300], torch.int64

## Dataset and DataLoader

In [ ]:
dataset_train = TensorDataset(train, y_train)
dataset_val = TensorDataset(val, y_val)

In [ ]:
BATCH_SIZE = 128
train_loader = DataLoader(dataset=dataset_train,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
val_loader = DataLoader(dataset=dataset_val,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

# Preview Images

In [ ]:
# preview first ten images
def print_ten_numbers(x: torch.Tensor, 
                      y: torch.Tensor):
    fig = plt.figure(figsize=(25, 10))  # (width, height) in inches
    for i in range(10):
        ax = fig.add_subplot(1,  # nrows
                             10,  # ncols
                             i+1, # index (1-based)
                             xticks=[],
                             yticks=[])
        image = x[i]  # [28, 28]
        label = y[i].item()  # int
        ax.imshow(X=image.squeeze().cpu(),
                  cmap='gray')
        ax.set_title(f"{label}")

x_batch, y_batch = next(iter(train_loader))   # x_batch: [128, 28, 28] torch.float32, y_batch: [128] torch.int64
print_ten_numbers(x=x_batch, y=y_batch)

# CNN Classifier

In [ ]:
class CNNClassifier(torch.nn.Module):
    # https://machinelearningknowledge.ai/pytorch-conv2d-explained-with-examples/

    def __init__(self, dropout_probability=0.3):
        super(CNNClassifier, self).__init__()
        
        # add dummy input channel: [batch_size, 28, 28] -> [batch_size, 1, 28, 28]
        self.unflatten = torch.nn.Unflatten(dim=1, unflattened_size=(1,28))
        
        self.layer1 = torch.nn.Sequential(
            # (batch_size, 1, 28, 28) -> (batch_size, 64, 28, 28)
            torch.nn.Conv2d(in_channels=1,  # Number of channels in the input image
                            out_channels=64,  # Number of channels produced by the convolution
                            kernel_size=3, #  Size of the convolving kernel
                            stride=1,  # Stride of the convolution. Default: 1
                            padding=1,  # Padding added to all four sides of the input. Default: 0
                           ),
            # (element-wise)
            torch.nn.ReLU(),
            
            # shape unchanged
            torch.nn.Conv2d(in_channels=64,
                            out_channels=64,
                            kernel_size=3,
                            stride=1,
                            padding=1),
            torch.nn.ReLU(),
            
            # [batch_size, 64, 28, 28] -> [batch_size, 128, 28, 28]
            torch.nn.Conv2d(in_channels=64,
                            out_channels=128,
                            kernel_size=3,
                            stride=1,
                            padding=1),
            torch.nn.ReLU(),
            
            # [batch_size, 128, 28, 28] -> [batch_size, 128, 14, 14]
            torch.nn.MaxPool2d(kernel_size=2, # the size of the window to take a max over
                               stride=2,  # the stride of the window. Default value is kernel_size
                              ),
            # (element-wise)
            torch.nn.Dropout(p=dropout_probability,  # probability of an element to be zeroed. Default: 0.5
                            ),
        )

        self.layer2 = torch.nn.Sequential(
            # shape unchanged
            torch.nn.Conv2d(in_channels=128,
                            out_channels=128,
                            kernel_size=3,
                            stride=1,
                            padding=1),
            torch.nn.ReLU(),
            
            # [batch_size, 128, 14, 14] -> [batch_size, 192, 14, 14]
            torch.nn.Conv2d(in_channels=128,
                            out_channels=192,
                            kernel_size=3,
                            stride=1,
                            padding=1),
            torch.nn.ReLU(),
            
            # [batch_size, 192, 14, 14] -> [batch_size, 192, 7, 7]
            torch.nn.MaxPool2d(kernel_size=2,
                               stride=2),
            torch.nn.Dropout(p=dropout_probability))

        self.layer3 = torch.nn.Sequential(
            # shape unchanged
            torch.nn.Conv2d(in_channels=192,
                            out_channels=192,
                            kernel_size=3,
                            stride=1,
                            padding=1),
            torch.nn.ReLU(),
            
            # [batch_size, 192, 7, 7] --> [batch_size, 192, 4, 4]
            torch.nn.MaxPool2d(kernel_size=2, 
                               stride=2, 
                               padding=1),  # default: 0
            torch.nn.Dropout(p=dropout_probability)
            )
        
        # [batch_size, 192, 4, 4] -> [batch_size, 2048]
        self.flatten = torch.nn.Flatten()  # for feed-forward 

        # [batch_size, 2048] --> [batch_size, 625]
        self.fc1 = torch.nn.Linear(in_features=4 * 4 * 192,
                                   out_features=625,
                                   bias=True)
        
        # [batch_size, 625] --> [batch_size, 10]
        self.fc2 = torch.nn.Linear(in_features=625,
                                   out_features=10,
                                   bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc1.weight)  # initialize weights (seems to make no difference in scores)
        torch.nn.init.xavier_uniform_(self.fc2.weight) 
        

    def forward(self, x: torch.Tensor) -> torch.Tensor:  # x: [batch_size, 28, 28]
        
        # CNN
        x_unflattened = self.unflatten(x)  # [batch_size, 1, 28, 28]
        output_layer_1 = self.layer1(x_unflattened)  # [batch_size, 128, 14, 14]
        output_layer_2 = self.layer2(output_layer_1)  # [batch_size, 192, 7, 7]
        output_layer_3 = self.layer3(output_layer_2)  # [batch_size, 192, 4, 4]
        flattened = self.flatten(output_layer_3)  # flattened to[batch_size, 3072]
        
        # FC
        output_fully_connected_1 = self.fc1(flattened)  # [batch_size, 625]
        output_fully_connected_2 = self.fc2(output_fully_connected_1)  # [batch_size, 10]

        return output_fully_connected_2

In [ ]:
# visualize the classifier
c_temp = CNNClassifier().to(DEVICE)
# to visualize with torchviz, we need some input that can pass through the model's forward() method.
predictions = c_temp(x_batch)
make_dot(predictions)

## Training

In [ ]:
LEARNING_RATE = 0.001

classifier = CNNClassifier().to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(),
                             lr = LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  # reduce learning rate when model stops improving on validation dataset 
                                                       mode='min',
                                                       factor=0.3,
                                                       patience=2,
                                                       min_lr=0.00000001,
                                                       verbose=True)

In [ ]:
def compute_metrics(classifier: CNNClassifier, 
                    loss_fn: Callable,
                    dataloader: torch.utils.data.DataLoader,
                   )->tuple[float, float, float]:
        
        losses = []
        predictions = []
        y_true = []
        
        for x_batch, y_batch in dataloader:
            pred_logits_batch = classifier(x_batch)  # [batch_size, 10]
            loss = loss_fn(pred_logits_batch,
                           y_batch)
            losses.extend([loss.item()] * len(x_batch))
            
            pred_batch = pred_logits_batch.argmax(dim=1)  # [batch_size]
            predictions.extend(pred_batch.tolist())
            y_true.extend(y_batch.tolist())
        
        avg_loss = np.mean(losses)
        correct = (np.array(predictions) == np.array(y_true))
        accuracy = correct.mean()
        f1_score = sklearn.metrics.f1_score(y_true=y_true, 
                                            y_pred=predictions,
                                            average='micro')  # multi-class problem
        
        return avg_loss, accuracy, f1_score

In [ ]:
df_metrics = pd.DataFrame(columns=['loss_train', 'accuracy_train', 'f1_train', 
                                   'loss_val', 'accuracy_val', 'f1_val'],
                          index=range(NUM_EPOCHS))

for epoch in tqdm(range(NUM_EPOCHS)):

    for batch, (x_train_batch, y_train_batch) in enumerate(train_loader):
        # x_train_batch: [batch_size, 28, 28] torch.float32
        # y_train_batch: [batch_size] torch.int64

        # switch to training mode mode (we might have been in evaluation mode)
        classifier.train()

        pred_train_batch = classifier(x_train_batch)  # [batch_size, 10]

        # clear existing gradients from previous batch
        optimizer.zero_grad()

        loss = loss_fn(pred_train_batch,
                       y_train_batch)  # [], .item() is e.g. 2.291177988052368

        # compute gradients (backpropagation), then apply gradients
        loss.backward()
        optimizer.step()
        
    # after each epoch, switch to evaluation mode, then evaluate without computing gradients
    classifier.eval()
    with torch.no_grad():
        loss_train, accuracy_train, f1_score_train = compute_metrics(classifier, loss_fn, train_loader)
        loss_val, accuracy_val, f1_score_val = compute_metrics(classifier, loss_fn, val_loader)

        df_metrics.iloc[epoch] = [loss_train, accuracy_train, f1_score_train,
                                  loss_val, accuracy_val, f1_score_val]
        
    scheduler.step(loss_val)
    print(f'Accuracy Validation after epoch {epoch}: {accuracy_val :.4f}  '
          f'(Train: {accuracy_train :.4f}) '
          f'LR = {optimizer.param_groups[0]["lr"]}\n')

# Evaluation

In [ ]:
df_metrics

In [ ]:
epochs = range(NUM_EPOCHS)

fig, ((ax1, ax2), (ax3, _)) = plt.subplots(nrows=2,
                                       ncols=2,
                                       figsize=(15,5),
                                          sharex=True)

# Plot and label the training and val loss values
ax1.plot(epochs, df_metrics['loss_train'], label='Training Loss')
ax1.plot(epochs, df_metrics['loss_val'], label='val Loss')
ax1.set_ylabel('Loss')
ax1.legend(loc='best')

# ... Accuracy
ax2.plot(epochs, df_metrics['accuracy_train'], label='Training Accuracy')
ax2.plot(epochs, df_metrics['accuracy_val'], label='val Accuracy')
ax2.set_ylabel('Accuracy')
ax2.legend(loc='best')

# ... F1-Score
ax3.plot(epochs, df_metrics['f1_train'], label='Training F1-Score')
ax3.plot(epochs, df_metrics['f1_val'], label='val F1-Score')
ax3.set_ylabel('F1-Score')
ax3.legend(loc='best')
ax3.set_xlabel('Epochs')
ax3.set_xticks(np.arange(0, 
                         NUM_EPOCHS))

plt.suptitle('Training and Validation Metrics')
plt.xlabel('Epochs')
plt.xticks(np.arange(0, 
                     NUM_EPOCHS))

plt.show()

## False Predictions

In [ ]:
# let's have a look at some of the misclassified images from the validation dataset
classifier.eval()
with torch.no_grad():
    y_pred_logits = classifier(val)
    y_pred = y_pred_logits.argmax(dim=1)
    correct = (y_pred == y_val).type(torch.FloatTensor)  # [6300] with either 1.0 or 0.0
   
ser_correct = pd.Series(correct)
print(ser_correct.value_counts())

In [ ]:
bad_indexes = ser_correct[ser_correct == 0.0].index.to_list()

val_misclassified = val[bad_indexes]  # [69, 28, 28]
y_val_misclassified = y_val[bad_indexes]
y_pred_misclassified = y_pred[bad_indexes]

In [ ]:
fig, axes = plt.subplots(nrows=math.ceil(len(val_misclassified) / 10),  # axes: np.array of shape (7, 10)
                         ncols=10,
                         figsize=(15,15),
                        )

for i in range(len(val_misclassified)):
    image = val_misclassified[i]
    y = y_val_misclassified[i].item()
    pred = y_pred_misclassified[i].item()
    
    ax = axes[i//10, i%10]
    
    ax.imshow(X=image.squeeze(dim=0).cpu(),
              cmap='gray')
    ax.set_xticks([]) 
    ax.set_yticks([]) 
    ax.set_title(f"✓{y} / ⚠ {pred}")

# Submission

## Predict Test Data

In [ ]:
# with the current model, predicting the whole test dataset in one batch
# runs into cuda memory problems
test.shape  # [28000, 28, 28]

In [ ]:
test_chunks: tuple[torch.Tensor, ...] = torch.split(test, 1000)

predicted_labels: list[int] = []
    
with torch.no_grad():
    for test_chunk in test_chunks:
        pred_logits = classifier(test_chunk)  # [chunk_size, 10]
        pred = pred_logits.argmax(dim=1)   # [chunk_size]
        predicted_labels.extend(pred.tolist())

assert len(predicted_labels) == len(test)

In [ ]:
df_pred = pd.DataFrame({'ImageId': range(1, len(test)+1),
                        'Label': predicted_labels})
df_pred['Label'].value_counts()

In [ ]:
df_pred.to_csv('submission.csv',
               index=False)

# Visualization of CNN Layers

## Image and Prediction

In [ ]:
n_example = 100

# let's look at an example image from the validation dataset:
x_example = val[n_example].detach()  # [28, 28]
plt.imshow(x_example.cpu(), cmap='gray')

In [ ]:
# predicted logits
classifier.eval()
with torch.no_grad():
    predicted_logits = classifier(x_example.unsqueeze(dim=0).to(DEVICE))  # [1, 10]
    for i in range(10):
        print(f'{i}: {round(predicted_logits[0][i].item(), 2) :5}')

## Outputs of CNN Layer 1

In [ ]:
# from [28, 28] to [batch_size, channel, 28, 28], i.e. [1, 1, 28, 28]
x_unsqueezed = x_example.unsqueeze(dim=0)
x_unsqueezed = x_unsqueezed.unsqueeze(dim=0)

classifier.eval()
with torch.no_grad():
    output_layer_1 = classifier.layer1(x_unsqueezed.to(DEVICE).detach())  # 1, 128, 14, 14
    print(output_layer_1.shape)
                                       

fig = plt.figure(figsize=(25, 10))  # (width, height) in inches
for i in range(40):  # can't show all 128, 40 is the limit here
    ax = fig.add_subplot(4,         # nrows
                         10,        # ncols
                         i+1,       # index (1-based)
                         xticks=[],
                         yticks=[])
    image = output_layer_1[0][i]  # (14, 14)
    ax.imshow(X=image.cpu(),
              cmap='gray')

### Show the outputs of CNN Layer 2

In [ ]:
classifier.eval()
with torch.no_grad():
    output_layer_2 = classifier.layer2(output_layer_1.to(DEVICE)).detach()  # [1, 192, 7, 7]

fig = plt.figure(figsize=(25, 5))  # (width, height) in inches
for i in range(80):
    ax = fig.add_subplot(
                 4,         # nrows
                 20,        # ncols
                 i+1,       # index (1-based)
                 xticks=[],
                 yticks=[])
    image = output_layer_2[0][i]  # (7, 7)
    ax.imshow(X=image.cpu(),
              cmap='gray')

### Show the outputs of CNN Layer 3

In [ ]:
classifier.eval()
with torch.no_grad():
    output_layer_3 = classifier.layer3(output_layer_2.to(DEVICE)).detach()  # [1, 192, 4, 4]
    
fig = plt.figure(figsize=(25, 5))  # (width, height) in inches
for i in range(150):
    ax = fig.add_subplot(
                 5,         # nrows
                 30,        # ncols
                 i+1,       # index (1-based)
                 xticks=[],
                 yticks=[])
    image = output_layer_3[0][i]  # (28, 28)
    ax.imshow(X=image.cpu(),
              cmap='gray')